In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("domainadaptationexperiments.csv").dropna()
df = pd.concat([df[df['training_method'] == 'FedSCDepth(Retrain)'], df[df['training_method'] == 'FedSCDepth']])
metrics = ['test_abs_rel', 'test_sq_rel', 'test_log10', 'test_rmse', 'test_rmse_log', 'test_a1', 'test_a2', 'test_a3']

In [15]:
df_full = df[df['test_region_type'] == 'full'].drop(columns=['test_region_type'])
df_full

,source_train_dataset,target_train_dataset,training_method,test_dataset,adaptation_state,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
72,KITTI,DDAD,FedSCDepth(Retrain),KITTI,before_adaptation,0.120,0.762,0.052,4.667,0.186,0.859,0.961,0.985
75,KITTI,DDAD,FedSCDepth(Retrain),DDAD,before_adaptation,0.246,5.862,0.123,23.383,0.414,0.579,0.808,0.893
78,KITTI,DDAD,FedSCDepth(Retrain),KITTI,after_adaptation,0.170,1.204,0.069,5.266,0.226,0.772,0.937,0.981
81,KITTI,DDAD,FedSCDepth(Retrain),DDAD,after_adaptation,0.175,3.970,0.085,19.397,0.303,0.736,0.887,0.941
96,DDAD,KITTI,FedSCDepth(Retrain),KITTI,before_adaptation,0.150,3.279,0.072,17.040,0.264,0.790,0.913,0.956
99,DDAD,KITTI,FedSCDepth(Retrain),DDAD,before_adaptation,0.364,8.693,0.229,26.120,0.742,0.436,0.617,0.699
102,DDAD,KITTI,FedSCDepth(Retrain),KITTI,after_adaptation,0.119,0.767,0.052,4.632,0.187,0.863,0.962,0.985
105,DDAD,KITTI,FedSCDepth(Retrain),DDAD,after_adaptation,0.216,4.896,0.108,21.308,0.366,0.636,0.846,0.915
12,KITTI,DDAD,FedSCDepth,KITTI,before_adaptation,0.120,0.762,0.052,4.667,0.186,0.859,0.961,0.985
15,KITTI,DDAD,FedSCDepth,DDAD,before_adaptation,0.246,5.862,0.123,23.383,0.414,0.579,0.808,0.893


In [3]:
def compute_difference_proportions(group):
    before_adaptation_group = group[group['adaptation_state'] == 'before_adaptation']
    before_adaptation_row = {metric: None for metric in metrics}
    if len(before_adaptation_group) > 0:
        before_adaptation_row = before_adaptation_group.iloc[0]
    after_adaptation_group = group[group['adaptation_state'] == 'after_adaptation']
    after_adaptation_row = {metric: None for metric in metrics}
    if len(after_adaptation_group) > 0:
        after_adaptation_row = after_adaptation_group.iloc[0]
    differences = {}
    for metric in metrics:
        differences[metric] = None
        if before_adaptation_row[metric] and after_adaptation_row[metric]:
            differences[metric] = (before_adaptation_row[metric] - after_adaptation_row[metric]) / before_adaptation_row[metric]
    return pd.Series(differences)

In [4]:
df_diff_prop = df_full.groupby(['source_train_dataset', 'target_train_dataset', 'training_method', 'test_dataset'], as_index=False).apply(compute_difference_proportions)
df_diff_prop

,source_train_dataset,target_train_dataset,training_method,test_dataset,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
0,DDAD,KITTI,FedSCDepth,DDAD,0.412088,0.425285,0.532751,0.173622,0.505391,-0.481651,-0.364668,-0.306152
1,DDAD,KITTI,FedSCDepth,KITTI,0.206667,0.757853,0.291667,0.729636,0.295455,-0.094937,-0.054765,-0.030335
2,DDAD,KITTI,FedSCDepth(Retrain),DDAD,0.406593,0.436788,0.528384,0.184227,0.506739,-0.458716,-0.371151,-0.309013
3,DDAD,KITTI,FedSCDepth(Retrain),KITTI,0.206667,0.766087,0.277778,0.728169,0.291667,-0.092405,-0.053669,-0.030335
4,KITTI,DDAD,FedSCDepth,DDAD,0.365854,0.414364,0.390244,0.249754,0.342995,-0.352332,-0.122525,-0.066069
5,KITTI,DDAD,FedSCDepth,KITTI,-0.825000,-1.938320,-0.596154,-0.521106,-0.440860,0.178114,0.060354,0.019289
6,KITTI,DDAD,FedSCDepth(Retrain),DDAD,0.288618,0.322757,0.308943,0.170466,0.268116,-0.271157,-0.097772,-0.053751
7,KITTI,DDAD,FedSCDepth(Retrain),KITTI,-0.416667,-0.580052,-0.326923,-0.128348,-0.215054,0.101281,0.024974,0.004061


In [5]:
df_diff_prop_avg = df_diff_prop.groupby(['source_train_dataset', 'target_train_dataset', 'training_method'], as_index=False)[metrics].mean()
df_diff_prop_avg

,source_train_dataset,target_train_dataset,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
0,DDAD,KITTI,FedSCDepth,0.309377,0.591569,0.412209,0.451629,0.400423,-0.288294,-0.209716,-0.168243
1,DDAD,KITTI,FedSCDepth(Retrain),0.306630,0.601438,0.403081,0.456198,0.399203,-0.275560,-0.212410,-0.169674
2,KITTI,DDAD,FedSCDepth,-0.229573,-0.761978,-0.102955,-0.135676,-0.048933,-0.087109,-0.031085,-0.023390
3,KITTI,DDAD,FedSCDepth(Retrain),-0.064024,-0.128648,-0.008990,0.021059,0.026531,-0.084938,-0.036399,-0.024845


In [6]:
df_diff_prop_avg_ddad_to_kitti = df_diff_prop_avg[df_diff_prop_avg['source_train_dataset'] == 'KITTI']
df_diff_prop_avg_ddad_to_kitti.sort_values(by=['test_abs_rel'],ascending=False)

,source_train_dataset,target_train_dataset,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
3,KITTI,DDAD,FedSCDepth(Retrain),-0.064024,-0.128648,-0.008990,0.021059,0.026531,-0.084938,-0.036399,-0.024845
2,KITTI,DDAD,FedSCDepth,-0.229573,-0.761978,-0.102955,-0.135676,-0.048933,-0.087109,-0.031085,-0.023390


In [7]:
df_diff_prop_avg_kitti_to_ddad = df_diff_prop_avg[df_diff_prop_avg['source_train_dataset'] == 'DDAD']
df_diff_prop_avg_kitti_to_ddad.sort_values(by=['test_abs_rel'],ascending=False)

,source_train_dataset,target_train_dataset,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
0,DDAD,KITTI,FedSCDepth,0.309377,0.591569,0.412209,0.451629,0.400423,-0.288294,-0.209716,-0.168243
1,DDAD,KITTI,FedSCDepth(Retrain),0.306630,0.601438,0.403081,0.456198,0.399203,-0.275560,-0.212410,-0.169674


In [8]:
df_diff_prop_avg_ciclic = df_diff_prop_avg.groupby(['training_method'], as_index=False)[metrics].mean()

In [9]:
df_diff_prop_avg_ciclic.sort_values(by=['test_abs_rel'],ascending=False)

,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
1,FedSCDepth(Retrain),0.121303,0.236395,0.197046,0.238628,0.212867,-0.180249,-0.124405,-0.097260
0,FedSCDepth,0.039902,-0.085205,0.154627,0.157977,0.175745,-0.187701,-0.120401,-0.095817


In [10]:
df_diff_prop_avg_ciclic.sort_values(by=['test_sq_rel'],ascending=False)

,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
1,FedSCDepth(Retrain),0.121303,0.236395,0.197046,0.238628,0.212867,-0.180249,-0.124405,-0.097260
0,FedSCDepth,0.039902,-0.085205,0.154627,0.157977,0.175745,-0.187701,-0.120401,-0.095817


In [11]:
df_diff_prop_avg_ciclic.sort_values(by=['test_rmse'],ascending=False)

,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
1,FedSCDepth(Retrain),0.121303,0.236395,0.197046,0.238628,0.212867,-0.180249,-0.124405,-0.097260
0,FedSCDepth,0.039902,-0.085205,0.154627,0.157977,0.175745,-0.187701,-0.120401,-0.095817


In [12]:
df_diff_prop_avg_ciclic.sort_values(by=['test_a1'],ascending=True)

,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
0,FedSCDepth,0.039902,-0.085205,0.154627,0.157977,0.175745,-0.187701,-0.120401,-0.095817
1,FedSCDepth(Retrain),0.121303,0.236395,0.197046,0.238628,0.212867,-0.180249,-0.124405,-0.097260


In [13]:
df_diff_prop_avg_ciclic.sort_values(by=['test_a2'],ascending=True)

,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
1,FedSCDepth(Retrain),0.121303,0.236395,0.197046,0.238628,0.212867,-0.180249,-0.124405,-0.097260
0,FedSCDepth,0.039902,-0.085205,0.154627,0.157977,0.175745,-0.187701,-0.120401,-0.095817


In [14]:
df_diff_prop_avg_ciclic.sort_values(by=['test_a3'],ascending=True)

,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
1,FedSCDepth(Retrain),0.121303,0.236395,0.197046,0.238628,0.212867,-0.180249,-0.124405,-0.097260
0,FedSCDepth,0.039902,-0.085205,0.154627,0.157977,0.175745,-0.187701,-0.120401,-0.095817
